In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
df=pd.read_csv("")

In [8]:
!pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 11.5 MB/s eta 0:00:00


In [9]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 59.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.4 MB/s eta 0:00:00


In [10]:
import random
import re
import string

import contractions

import nltk
from nltk import pos_tag
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

from transformers import TFRobertaModel, RobertaTokenizerFast
from keras.utils.vis_utils import plot_model
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline

from keras.models import Model
from keras.layers import Dense, Dropout, Input
from keras.optimizers import Adam
from keras.losses import CategoricalCrossentropy
from keras.metrics import CategoricalAccuracy
from keras.callbacks import EarlyStopping, ModelCheckpoint


from tabulate import tabulate
from wordcloud import WordCloud

import warnings
warnings.filterwarnings('ignore')

In [11]:
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('words')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [3]:
df.head

<bound method NDFrame.head of                                                   title  \
0                        Envy to other is swallowing me   
1     Nothin outta the ordinary. Paradise. Job stres...   
2     Almost 49 and the chasm of emptiness has never...   
3                                       I’m happy again   
4     Is it possible to recover from such a traumati...   
...                                                 ...   
3118                 Positive relief ideas with stress?   
3119                                          Breakdown   
3120  I feel as if I actually died a long time ago a...   
3121  Is it wierd that i have imaginary friends at t...   
3122                              Head and Eye Pressure   

                                                   body  \
0     Im from developingcountry, Indonesia , and for...   
1     Um hello ....well many can relate im sure. Aft...   
2     I’ve been diagnosed severe bi polar where you ...   
3     After my closest fr

In [4]:
del df['title']
del df['body']
df.rename(columns={'Body_Title':'content'},inplace=True)
df.rename(columns={'label':'sentiment'},inplace=True)


In [43]:

train = df.copy()

x = np.array(train['content'].values)
y = np.array(train['sentiment'].values)


encoder = LabelEncoder()
y = encoder.fit_transform(y)
labels = np.unique(encoder.inverse_transform(y))


x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=True, random_state=42, stratify=y)


tf1 = TfidfVectorizer(analyzer='word', max_features=1000, ngram_range=(1,3))
x_train = tf1.fit_transform(x_train).toarray()
x_test = tf1.transform(x_test).toarray()



In [44]:
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import GridSearchCV
import xgboost as xgb



rf_param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 5, 10],
    'random_state': [42]
}

xgb_param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1],
    'random_state': [42]
}


rf_grid_search = GridSearchCV(RandomForestClassifier(), param_grid=rf_param_grid, cv=3)
xgb_grid_search = GridSearchCV(xgb.XGBClassifier(), param_grid=xgb_param_grid, cv=3)


rf_grid_search.fit(x_train, y_train)
xgb_grid_search.fit(x_train, y_train)


best_rf_classifier = rf_grid_search.best_estimator_
best_xgb_classifier = xgb_grid_search.best_estimator_


voting_classifier = VotingClassifier(estimators=[
    ('rf', best_rf_classifier),
    ('xgb', best_xgb_classifier)
], voting='hard')

voting_classifier.fit(x_train, y_train)

y_pred = voting_classifier.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Voting Classifier Accuracy: {accuracy:.2f}")

best_rf_params = best_rf_classifier.get_params()
best_xgb_params = best_xgb_classifier.get_params()

print("Best Random Forest Parameters:", best_rf_params)
print("Best XGBoost Parameters:", best_xgb_params)

Voting Classifier Accuracy: 0.93
Best Random Forest Parameters: {'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 42, 'verbose': 0, 'warm_start': False}
Best XGBoost Parameters: {'objective': 'binary:logistic', 'use_label_encoder': None, 'base_score': None, 'booster': None, 'callbacks': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': None, 'early_stopping_rounds': None, 'enable_categorical': False, 'eval_metric': None, 'feature_types': None, 'gamma': None, 'gpu_id': None, 'grow_policy': None, 'importance_type': None, 'interaction_constraints': None, 'learning_rate': 0.1, 'max_bin': None, 'max_cat_threshold': None, 'max_cat_to_onehot': None, 'max_delta_step': 

In [45]:
import joblib
joblib.dump(voting_classifier, 'voting_classifier_model.pkl')

['voting_classifier_model.pkl']

In [70]:

s=df[df['sentiment']==0].iloc[0:5]
s1=df[df['sentiment']==1].iloc[0:5]
print(s)
print(s1)

                                              content  sentiment
3   I’m happy again After my closest friend left m...          0
18  Screen Time Negatives I feel like these screen...          0
24  End of Semester Ah!!!! I’m almost done with th...          0
36  The love of my life, mother of my child, fianc...          0
38  Chat? Anyone just wanna chat in this thread? C...          0
                                             content  sentiment
0  Envy to other is swallowing me Im from develop...          1
1  Nothin outta the ordinary. Paradise. Job stres...          1
2  Almost 49 and the chasm of emptiness has never...          1
4  Is it possible to recover from such a traumati...          1
5  I’m finding it hard to find a reason to get up...          1


In [81]:
para="it was a great day . i am really happy"

para=[para]
vectorized = tf1.transform(para).toarray()

predictions=voting_classifier.predict(vectorized)
print(predictions)

[0]
